### Install Pre-requisites

In [1]:
%%bash
python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


### Import Modules

In [2]:
import json
import math
import random
import time

import spacy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext.data as td

SEED = 2020

### Create Dataset

In [3]:
!unzip coqa-train-v1.0.zip

Archive:  coqa-train-v1.0.zip
replace coqa-train-v1.0.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: coqa-train-v1.0.json    


In [4]:
def iteratively_add_conv_history(psg_with_turns, fields):
    psg = psg_with_turns["story"]
    total_turns = len(psg_with_turns["questions"])
    dataset = []
    for q in range(total_turns):
        question = psg_with_turns["questions"][q]["input_text"]
        answer = psg_with_turns["answers"][q]["input_text"]
        psg += (" chiknichameli " + question)
        dataset.append( (psg, answer, fields) )
        psg += (" chiknachamela " + answer)
    return dataset

def create_dataset_from_raw(alldata, fields):
    dataset = []
    for d in alldata["data"]:
        dataset.extend( iteratively_add_conv_history(psg_with_turns=d, fields=fields) )
    return dataset


spacy_en = spacy.load('en')
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]


SRC = td.Field(sequential=True, tokenize=tokenize_en, init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)
TRG = td.Field(sequential=True, tokenize=tokenize_en, is_target=True, init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)
fields = [("src", SRC), ("trg", TRG)]

alldata = json.load(  open("coqa-train-v1.0.json", "r")  )
example = [  td.Example.fromlist([q,a], f) for q,a,f in create_dataset_from_raw(alldata=alldata, fields=fields)  ] 

ds = td.Dataset(example, fields)

In [5]:
(train_data, valid_data) = ds.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [6]:
SRC.build_vocab(train_data, min_freq = 7)
TRG.build_vocab(train_data, min_freq = 7)

In [7]:
BATCH_SIZE = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def sk(x):
    return len(  next(x.question)  )

# train_iter, valid_iter = td.BucketIterator.splits( (train_data, valid_data), 
#                                                               batch_size=BATCH_SIZE, 
#                                                               device=device )

train_iter = td.BucketIterator( train_data, 
                                                batch_size=BATCH_SIZE, 
                                                device=device )
valid_iter = td.BucketIterator( valid_data, 
                                                batch_size=BATCH_SIZE, 
                                                device=device )

### Create Encoder Model

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        return outputs, hidden

### Create Attention Model

In [9]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        attention = self.v(energy).squeeze(2)
        return F.softmax(attention, dim=1)

### Create Decoder Model

In [10]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        a = self.attention(hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        return prediction, hidden.squeeze(0)

### Seq To Seq Model

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1

        return outputs

### Initialize model

In [12]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 64
DEC_EMB_DIM = 64
ENC_HID_DIM = 128
DEC_HID_DIM = 128
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

### Initialize Weights As Given In The Paper

In [13]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(64480, 64)
    (rnn): GRU(64, 128, bidirectional=True)
    (fc): Linear(in_features=256, out_features=128, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=384, out_features=128, bias=True)
      (v): Linear(in_features=128, out_features=1, bias=False)
    )
    (embedding): Embedding(4430, 64)
    (rnn): GRU(320, 128)
    (fc_out): Linear(in_features=448, out_features=4430, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

### Count Parameters

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,803,406 trainable parameters


### Initialize Training Components

In [15]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### Create Train Function

In [16]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

### Create Evaluation Function

In [17]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            output = model(src, trg, 0)
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

### Time Keeper Function

In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Start Training

In [19]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):    
    start_time = time.time()
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 9m 59s
	Train Loss: 4.547 | Train PPL:  94.382
	 Val. Loss: 4.454 |  Val. PPL:  85.944
Epoch: 02 | Time: 10m 18s
	Train Loss: 4.126 | Train PPL:  61.920
	 Val. Loss: 4.045 |  Val. PPL:  57.101
Epoch: 03 | Time: 10m 28s
	Train Loss: 3.789 | Train PPL:  44.230
	 Val. Loss: 3.934 |  Val. PPL:  51.103
Epoch: 04 | Time: 10m 50s
	Train Loss: 3.618 | Train PPL:  37.280
	 Val. Loss: 3.910 |  Val. PPL:  49.884
Epoch: 05 | Time: 10m 44s
	Train Loss: 3.491 | Train PPL:  32.808
	 Val. Loss: 3.873 |  Val. PPL:  48.100
Epoch: 06 | Time: 10m 46s
	Train Loss: 3.364 | Train PPL:  28.913
	 Val. Loss: 3.901 |  Val. PPL:  49.453
Epoch: 07 | Time: 10m 46s
	Train Loss: 3.264 | Train PPL:  26.142
	 Val. Loss: 3.929 |  Val. PPL:  50.835
Epoch: 08 | Time: 10m 51s
	Train Loss: 3.165 | Train PPL:  23.681
	 Val. Loss: 3.952 |  Val. PPL:  52.046
Epoch: 09 | Time: 10m 43s
	Train Loss: 3.084 | Train PPL:  21.837
	 Val. Loss: 3.990 |  Val. PPL:  54.066
Epoch: 10 | Time: 10m 46s
	Train Loss: 3.000 | 

### Reload model and test again

In [20]:
model.load_state_dict(torch.load('tut2-model.pt'))
test_loss = evaluate(model, valid_iter, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.873 | Test PPL:  48.109 |


### Run the official Evaluation Script
(Check official leaderboard [here](https://stanfordnlp.github.io/coqa/))

In [21]:
# Create preds file
alldata = json.load(  open("coqa-dev-v1.0.json", "r")  )
example = [  td.Example.fromlist([q,a], f) for q,a,f in create_dataset_from_raw(alldata=alldata, fields=fields)  ] 

test_ds = td.Dataset(example, fields)

SRC.build_vocab(test_ds, min_freq = 5)
TRG.build_vocab(test_ds, min_freq = 5)

test_iter = td.BucketIterator( test_ds, 
                                                batch_size=BATCH_SIZE, 
                                                device=device )

In [22]:
!python evaluate-v1.0.py --data-file "coqa-train-v1.0.json" --pred-file <path_to_predictions>

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `python evaluate-v1.0.py --data-file "coqa-train-v1.0.json" --pred-file <path_to_predictions>'
